In [6]:
import tarfile
import os
directory = 'temp'
if not os.path.exists(directory):
    os.makedirs(directory)
tar = tarfile.open("modelgame", "w:gz")
tar.add("beta.h5")
tar.close()

In [9]:
import tarfile
path = 'modelgame'
#m1 = importlib.import_module('kerasmodel')
tar = tarfile.open(path, "r:gz")
text_info = [tarinfo for tarinfo in tar]
for tarinfo in tar:
    tar.extract(tarinfo)
tar.close()

In [10]:
text_info

[<TarInfo 'beta.h5' at 0x10cc96e20>]

In [4]:
import tempfile
#fod, path = tempfile.mkstemp(suffix='.h5')
fod, path = tempfile.mkstemp(prefix='kerasmodel')

In [5]:
fod

59

In [6]:
path

'/var/folders/2r/syzzntgn7hj2cscbl18mqxf80000gn/T/kerasmodelczj3_n8d'

In [27]:
import tensorflow.keras.models as krm

def load_model(path):
    model = krm.load_model(path)
    return model


def get_tmp_path():
    _ , path = tempfile.mkstemp(suffix='.h5')
    return path

def load_model_from_BytesIO(model_bytesio):
    """ Load a model from a BytesIO object. """
    path = get_tmp_path()
    with open(path, 'wb') as fh:
        fh.write(model_bytesio)
        fh.flush()

    return load_model(path)

In [59]:
os.path.join('dö','nu')


'dö/nu'

In [8]:
path = '879fa112-c861-4cb1-a25d-775153e5b548'
load_model_from_BytesIO(path)

TypeError: a bytes-like object is required, not 'str'

In [ ]:
from io import BytesIO
uploaded_seed = request.files['seed']
a = BytesIO()
a.seek(0, 0)
uploaded_seed.seek(0)
a.write(uploaded_seed.read()) 
model = self.control.helper.load_model_from_BytesIO(a.getbuffer())

In [10]:
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential

def create_seed_model():
    # input image dimensions
    img_rows, img_cols = 28, 28
    input_shape = (img_rows, img_cols, 1)
    num_classes = 10

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [115]:
model = create_seed_model()
#krm.load_model('')

In [121]:
import os
# Defining model as dict: model['kerasmodel'], model[]
def save_model(model, path=None):

    #if not path:
    _ , tar_path = tempfile.mkstemp(suffix='.tar.gz')

    _, weights_path = tempfile.mkstemp(suffix='.h5')
    # saving model weights
    model.save_weights(weights_path)

    import tarfile
    tar = tarfile.open(tar_path, "w:gz")
    tar.add(weights_path,'weights.h5')
    tar.add('kerasmodel.py')
    tar.close()
    return tar_path

def serialize_model_to_BytesIO(model):
    outfile_name = save_model(model)

    from io import BytesIO
    a = BytesIO()
    a.seek(0, 0)
    with open(outfile_name, 'rb') as f:
        a.write(f.read())
    os.unlink(outfile_name)
    return a

In [38]:
_ , path = tempfile.mkstemp()
outfile_name = path

from io import BytesIO
a = BytesIO()
a.seek(0, 0)
names = ['879fa112-c861-4cb1-a25d-775153e5b548', 'beta.h5']
for name in names:
    with open(name, 'rb') as f:
        a.write(f.read())
os.unlink(outfile_name)


In [ ]:
import tensorflow.keras.models as krm


def get_tmp_path():
    _ , path = tempfile.mkstemp(suffix='.h5')
    return path

def load_model_from_BytesIO(model_bytesio):
    """ Load a model from a BytesIO object. """
    path = get_tmp_path()
    with open(path, 'wb') as fh:
        fh.write(model_bytesio)
        fh.flush()

    return load_model(path)

In [122]:
foi = serialize_model_to_BytesIO(model)

In [123]:
from io import BytesIO

a = BytesIO()
a.seek(0, 0)
foi.seek(0)
#uploaded_seed.seek(0)
#a.write(uploaded_seed.read()) 
a.write(foi.read()) 

model2 = load_model_from_BytesIO(a.getbuffer())

tarinfo:  <TarInfo 'weights.h5' at 0x136447ae0>
tarinfo:  <TarInfo 'kerasmodel.py' at 0x136447c80>


In [35]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [78]:
def load_model(path="model"):
    import importlib
    import tarfile
    tempf = tempfile.TemporaryDirectory()
    tar = tarfile.open(path, "r:gz")
    for tarinfo in tar:
        print("tarinfo: ", tarinfo)
        tar.extract(tarinfo, tempf.name)
    tar.close()
    #m1 = importlib.import_module(os.path.join(tempf.name,'kerasmodel'))
    import importlib.util
    spec = importlib.util.spec_from_file_location("create_seed_model", os.path.join(tempf.name,"kerasmodel.py"))
    foo = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(foo)
    #foo.MyClass()
    model = foo.create_seed_model()
    model.load_weights('weights.h5')
    return model

In [79]:
from io import BytesIO
outfile_name = 'beta'
a = BytesIO()
a.seek(0, 0)
with open(outfile_name, 'rb') as f:
    a.write(f.read())
#os.unlink(outfile_name)

In [80]:
model2 = load_model_from_BytesIO(a.getbuffer())

tarinfo:  <TarInfo 'weights.h5' at 0x1363cac80>
tarinfo:  <TarInfo 'kerasmodel.py' at 0x1363cae20>


In [81]:
model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)              

In [63]:
r =tempfile.TemporaryDirectory()

In [65]:
r.name

'/var/folders/2r/syzzntgn7hj2cscbl18mqxf80000gn/T/tmpy5w5jvj0'

In [105]:
import tarfile
tar_path = 'nygil'
tar = tarfile.open(tar_path, "w:gz")
tar.add('modelgame')
tar.add('weights.h5','weights2.h5')
#tar.add('kerasmodel.py')
tar.close()

In [108]:
import importlib.util
import tarfile
#tempf = tempfile.TemporaryDirectory()
tar = tarfile.open(tar_path, "r:gz")
tempf_name ='roligtnamn'
for tarinfo in tar:
    print("tarinfo: ", tarinfo.name)
    tar.extract(tarinfo, os.path.join(tempf_name,tarinfo.name))
tar.close()


tarinfo:  modelgame


NotADirectoryError: [Errno 20] Not a directory: 'roligtnamn/modelgame/modelgame'

In [113]:
old_path = 'beta'
_, temp_path = tempfile.mkstemp(suffix='.py')
tempf = tempfile.TemporaryDirectory()

tar = tarfile.open(old_path, "r:gz")
for tarinfo in tar:
    print("tarinfo: ", tarinfo)
    if tarinfo.name == 'kerasmodel.py':
        temp_path = os.path.join(tempf.name,tarinfo.name)
        tar.extract(tarinfo, tempf.name)
tar.close()

tarinfo:  <TarInfo 'weights.h5' at 0x1364476d0>
tarinfo:  <TarInfo 'kerasmodel.py' at 0x136447600>


In [114]:
temp_path

'/var/folders/2r/syzzntgn7hj2cscbl18mqxf80000gn/T/tmpu89se4gy/kerasmodel.py'

In [88]:
tar = tarfile.open(tar_path, "w:gz")
tar.add('weights.h5')
tar.close()


In [98]:
archive = tarfile.open(tar_path, "w|gz")
archive.add('weights.h5', arcname='weights.h5')
archive.close()


In [126]:
"isla.py".endswith(".py")

True

In [ ]:
os.walk()